In [16]:
from argparse import ArgumentParser
import os
import sys
import time

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import shapely
import xarray as xr

from pyproj import Proj, transform

# from matplotlib import colors
from pandarallel import pandarallel

from reaches import *
from utils import *

### Parse arguments

In [2]:
# FOR NOW, SET
width_set = 'min'

# Control flow
if width_set == 'mean':
    width = 'WidthM'
    binn = 'Bin'
elif width_set == 'min':
    width = 'WidthM_Min'
    binn = 'Bin_Min'
elif width_set == 'max':
    width = 'WidthM_Max'
    binn = 'Bin_Max'
else:
    print('Invalid width option specified, exiting.')
    # sys.exit()

In [3]:
huc2 = '01' ### SET THIS
data_path = '/nas/cee-water/cjgleason/fiona/data/PIXC_v2_0_HUC2_' + huc2
# save_dir =

### Pixel Cloud

In [4]:
# Get job index
# slurm = int(os.environ['SLURM_ARRAY_TASK_ID'])
index = 106

In [5]:
huc2 = '01'
file_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/data/PIXC_v2_0_HUC2_' + huc2 + '_filtered.json'
data = open_json(file_path)

In [6]:
file_name = data[index]

In [7]:
# Get data for this tile
granule_name = file_name[:-3]
tile_name = file_name[20:28]
pass_num = int(file_name[20:23])

print(granule_name)

SWOT_L2_HR_PIXC_002_007_235R_20230811T080638_20230811T080649_PGC0_01


#### Read in PIXC

In [8]:
# Set PIXC filepath
pixc_path = os.path.join(data_path, file_name)

In [9]:
# Read in pixel group
ds_PIXC = xr.open_mfdataset(paths=pixc_path, group = 'pixel_cloud', engine='h5netcdf')

In [11]:
ds_TVP = xr.open_mfdataset(paths=pixc_path, group = 'tvp', engine='h5netcdf')

In [36]:
x = ds_TVP.x.to_dataframe()['x'].to_list()
y = ds_TVP.y.to_dataframe()['y'].to_list()
z = ds_TVP.z.to_dataframe()['z'].to_list()

In [37]:
# Zip the coordinates into a point object and convert to a GeoDataFrame
geometry = [Point(xyz) for xyz in zip(x, y, z)]
# df = GeoDataFrame(df, geometry=geometry)

# # Aggregate these points with the GroupBy
# df = df.groupby(['entity_id'])['geometry'].apply(lambda x: LineString(x.tolist()))
# df = GeoDataFrame(df, geometry='geometry')

In [41]:
LineString(geometry).coords[0]

(1913004.408472421, -4746645.640884261, 5159664.705711862)

In [ ]:
geometry

In [22]:
# Define ECEF to geodetic conversion function using pyproj
def geodetic_to_ecef(lat, lon, alt):
    # WGS84 datum
    wgs84 = Proj(init="epsg:4326")  # Lat, Lon, Alt (WGS84)
    ecef = Proj(init="epsg:4978")   # ECEF (meters)
    
    # Convert geodetic (lat, lon, alt) to ECEF (x, y, z)
    x, y, z = transform(wgs84, ecef, lon, lat, alt)
    return np.array([x, y, z])

# Function to calculate slant range from satellite to river point
def calculate_slant_range(satellite_position, river_points):
    slant_ranges = []
    
    for point in river_points:
        # Convert river point to ECEF
        river_ecef = geodetic_to_ecef(point[0], point[1], point[2])  # lat, lon, alt
        
        # Calculate the Euclidean distance (slant range) between satellite and river point
        slant_range = np.linalg.norm(satellite_position - river_ecef)
        slant_ranges.append(slant_range)
    
    return slant_ranges

In [23]:
# Example satellite position in ECEF (x, y, z) in meters
satellite_position = np.array([2000000, 3000000, 4000000])  # Example position

# Example river points (lat, lon, alt) in degrees and meters
# (lat, lon, alt) -> Latitude, Longitude, and Elevation (in meters)
river_points = [
    (37.7749, -122.4194, 30),  # Point 1 (San Francisco)
    (36.7783, -119.4179, 50),  # Point 2 (Central California)
    (34.0522, -118.2437, 10)   # Point 3 (Los Angeles)
]

In [24]:
# Calculate slant ranges for each river point
slant_ranges = calculate_slant_range(satellite_position, river_points)

# Output slant ranges
for idx, range_ in enumerate(slant_ranges):
    print(f"Slant range for river point {idx + 1}: {range_:.2f} meters")

Slant range for river point 1: 8653584.89 meters
Slant range for river point 2: 8716821.72 meters
Slant range for river point 3: 8897216.79 meters


/work/pi_cjgleason_umass_edu/.conda/envs/narrowPIXC/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/work/pi_cjgleason_umass_edu/.conda/envs/narrowPIXC/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_125220/351315837.py:8: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgr